In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
input_hcp_dwi='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/DWI_Baseline.nrrd'

input_hcp_t1='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution0/StrippedT1_125.nrrd'
input_hcp_t2='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution1/StrippedT2_125.nrrd'

input_hcp_brainmask='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution2/DWIBrainMask.nrrd'

OutputDir='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/2_SRWF/test_tune_parameters/outImageFiles'
MatlabFilesDir='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/2_SRWF/test_tune_parameters/matlabFiles'

In [3]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
#PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release_20150715/bin'
PROGRAM_PATHS = '/scratch/BS/release-BSR2/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

In [4]:
# 
# READ input DWI scan using pynrrd
#
import nrrd

class nrrdDWIHeader:
    """A helper class for manipulating header information
    from a nrrd DWI data set into a nibabel compliant
    format"""
    def __init__(self,pynrrdDataModel):
        self.modality=None                      #Matches NRRD File
        self.global_BValue=None                 #Matches NRRD File
        self.gradientUnormalizedVectors=[None]  #Matches NRRD File

        self.gradientIndex=-1
        self.gradientBValues=[None]             #Computed
        self.gradientVectors=[None]             #Computed
        self._ConvertNrrdToNibabelDWIDataModel(pynrrdDataModel)

    def Print(self):
        print("global_BValue {0}".format(self.global_BValue) )
        print("modality {0}".format(self.modality) )
        print("gradientBValues {0}".format(self.gradientBValues) )
        print("gradientDirections {0}".format(self.gradientUnormalizedVectors) )

    def _getGradientStorageIndex(self,pynrrdDataModel):
        """The 4D index that has separate gradients.
        all other directions are the spatial dimensions."""
        gradient_index=-1;
        centerings=pynrrdDataModel['centerings']
        numdwidims=len(centerings)
        for test_index in range(0,numdwidims):
            if centerings[test_index] not in ['cell']:
                gradient_index=test_index
        self.gradientIndex = gradient_index

    def _ExtractGlobalBValue(self, pyNrrdKVUnknownMetaData):
        globalBValueString=pyNrrdKVUnknownMetaData.get(u'DWMRI_b-value','0').lstrip().rstrip()
        self.global_BValue=float( globalBValueString )

    def _ExtractUnormalizedBValues(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        """Unnormalized data values from the nrrd files, where the
        magnitude is reflective of the scale relative to the global_BValue"""
        self._getGradientStorageIndex(pynrrdDataModel)
        numGradients = pynrrdDataModel[u'sizes'][self.gradientIndex]
        self.gradientUnormalizedVectors= np.array( [ [ None, None, None ] for x in range(0,numGradients) ] )
        gvec_Fields=pyNrrdKVUnknownMetaData.copy() #Clone so we can remove items
        for k,v in gvec_Fields.iteritems():
            if k.startswith("DWMRI_gradient_"):
                index=int(k.split('_')[2])
                self.gradientUnormalizedVectors[index,:]=np.array([ float(x) for x in v.rstrip().lstrip().split() ],
                                                    copy=True, order='C', ndmin=1)
                pyNrrdKVUnknownMetaData.pop(k)


    def _ExtractDWIModality(self,pyNrrdKVUnknownMetaData):
        self.modality=pyNrrdKVUnknownMetaData.get("modality","UNKNOWN").lstrip().rstrip()

    def _ComputeNormalizedBvecBVals(self,pyNrrdKVUnknownMetaData,pynrrdDataModel):
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ExtractUnormalizedBValues(pyNrrdKVUnknownMetaData,pynrrdDataModel)
        self._ComputeNormalizedGradientsAndBValues()

    def _ComputeNormalizedGradientsAndBValues(self):
        """Following conventions of NRRD format, expand
        to normalized vectors and estimate BValues
        :return: void
        """
        numGradients=len( self.gradientUnormalizedVectors )

        self.gradientVectors = np.array(self.gradientUnormalizedVectors)
        self.gradientBValues = np.array( [ self.global_BValue ] * numGradients )
        for index in range(0,numGradients):
            gv = self.gradientUnormalizedVectors[index]
            norm=np.linalg.norm(gv)
            if norm < 1e-2:
                self.gradientVectors[index] = gv * 0.0
                self.gradientBValues[index] = 0.0
            elif ( abs( 1.0-norm ) > 1e-4 ): # Avoid rescaling if norm is almost one
                self.gradientVectors[index] = gv/norm
                b_i = self.global_BValue * (norm**2) # norm = sqrt(b_i/b_max)
                self.gradientBValues[index] = float("{0:.1f}".format(b_i))

    def _ConvertNrrdToNibabelDWIDataModel(self, pynrrdDataModel):
        pyNrrdKVUnknownMetaData=pynrrdDataModel['keyvaluepairs']
        self._ExtractDWIModality(pyNrrdKVUnknownMetaData)
        self._ExtractGlobalBValue(pyNrrdKVUnknownMetaData)
        self._ComputeNormalizedBvecBVals(pyNrrdKVUnknownMetaData,pynrrdDataModel)

def ReadNAMICDWIFromNrrd(filename):
    nrrd_dwi_data,nrrd_dwi_header=nrrd.read(filename)
    nibabelDataModelDWI=nrrdDWIHeader(nrrd_dwi_header)
    nrrd_dwi_bvec=nibabelDataModelDWI.gradientVectors
    nrrd_dwi_bval=nibabelDataModelDWI.gradientBValues
    gradient_index=nibabelDataModelDWI.gradientIndex
    return (nrrd_dwi_data, nrrd_dwi_header, nrrd_dwi_bvec, nrrd_dwi_bval, gradient_index)

# Create edge map image from T1/T2 based on masked quantile

In [10]:
oldmethod = False

In [11]:
mgi_fn = os.path.join(OutputDir,'mgi.nrrd') # Maximum gradient image
edgemap_fn = os.path.join(OutputDir,'edgeMap.nrrd') # Maximum gradient image
BRIANSToolsPath = "/scratch/BS/release-BSR2/bin"

if(not oldmethod):
    !{BRIANSToolsPath}/GenerateEdgeMapImage \
    --inputMRVolumes {input_hcp_t1},{input_hcp_t2} \
    --inputMask {input_hcp_brainmask} \
    --outputEdgeMap {edgemap_fn} \
    --outputMaximumGradientImage {mgi_fn} \
    --minimumOutputRange 1 \
    --maximumOutputRange 100 \
    --lowerPercentileMatching 0.25 \
    --upperPercentileMatching 0.999
else:
    MGI = GenerateSummedGradientImage() # Maximum Gradient Image
    MGI.inputs.inputVolume1 = input_hcp_t1
    MGI.inputs.inputVolume2 = input_hcp_t2
    MGI.inputs.MaximumGradient = True
    MGI.inputs.outputFileName = mgi_fn
    print(MGI.cmdline)
    MGI.run()

Reading image: /scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution0/StrippedT1_125.nrrd
Reading image: /scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/HCPWorkflow_CACHE_105115/PreprocessingWorkflow_CACHE_105115/ResampleToAlignedDWIResolution/mapflow/_ResampleToAlignedDWIResolution1/StrippedT2_125.nrrd
Generating maximum gradient image...
[LowerQuantile UpperQuantile] = [0.25 0.999]
[minOutputRange maxOutputRange] [1 100]
Generating maximum gradient image took 0.165555 s.


In [ ]:
mgi = sitk.ReadImage(mgi_fn)

totalStats = sitk.StatisticsImageFilter()
totalStats.Execute(mgi)
print(totalStats.GetMaximum())
print(totalStats.GetMinimum())

myshow(mgi)

In [ ]:
if(not oldmethod):
    edgemap = sitk.ReadImage(edgemap_fn)
    totalStats = sitk.StatisticsImageFilter()
    totalStats.Execute(edgemap)
    print(totalStats.GetMaximum())
    print(totalStats.GetMinimum())
    myshow(edgemap)
else:
    mgi = sitk.Cast(mgi,sitk.sitkFloat32)
    # compute 1/mgi
    div = sitk.DivideImageFilter()
    edgemap = div.Execute(1.0,mgi)
    totalStats = sitk.StatisticsImageFilter()
    totalStats.Execute(edgemap)
    print(totalStats.GetMaximum())
    print(totalStats.GetMinimum())
    myshow(edgemap)

In [ ]:
edge_arr = sitk.GetArrayFromImage(edgemap)

import scipy.io
scipy.io.savemat(os.path.join(MatlabFilesDir,'edgemask_tmp.mat'), mdict={'edgemask': edge_arr})

In [ ]:
mgi_smooth = sitk.SmoothingRecursiveGaussian(mgi,1.25)
myshow(mgi_smooth)

In [ ]:
div = sitk.DivideImageFilter()
edgemap_s = div.Execute(1.0,mgi_smooth)
myshow(edgemap_s)

totalStats = sitk.StatisticsImageFilter()
totalStats.Execute(edgemap_s)
print(totalStats.GetMaximum())
print(totalStats.GetMinimum())
print(totalStats.GetMean())

edge_smooth_arr = sitk.GetArrayFromImage(edgemap_s)

print(edge_smooth_arr.dtype)
print(edge_smooth_arr.shape)
print(np.amax(edge_smooth_arr))
print(np.amin(edge_smooth_arr))
print(np.mean(edge_smooth_arr))

edge_smooth_arr = np.clip(edge_smooth_arr,np.amin(edge_smooth_arr),1.0)

print("-----")
print(edge_smooth_arr.dtype)
print(edge_smooth_arr.shape)
print(np.amax(edge_smooth_arr))
print(np.amin(edge_smooth_arr))
print(np.mean(edge_smooth_arr))

scipy.io.savemat(os.path.join(MatlabFilesDir,'edgemask_tmp_s.mat'), mdict={'edgemask': edge_smooth_arr})

In [ ]:
edgemap_smooth = sitk.SmoothingRecursiveGaussian(edgemap,2)
myshow(edgemap_smooth)

totalStats = sitk.StatisticsImageFilter()
totalStats.Execute(edgemap_smooth)
print(totalStats.GetMaximum())
print(totalStats.GetMinimum())
print(totalStats.GetMean())

'''
def NormalizeBetweenZeroAndOne(image):
    image_double = sitk.Cast(image, sitk.sitkFloat32)
    # new range
    newMax=1.0
    newMin=0
    # Find old range
    totalStats = sitk.StatisticsImageFilter()
    totalStats.Execute(image_double)
    oldMax=totalStats.GetMaximum()
    oldMin=totalStats.GetMinimum()
    f=(newMax-newMin)/(oldMax-oldMin)
    Normalized_image = (image_double-oldMin)*f+newMin
    return Normalized_image

edgemap_smooth = NormalizeBetweenZeroAndOne(edgemap_smooth)
'''

edge_smooth_arr = sitk.GetArrayFromImage(edgemap_smooth)

print(edge_smooth_arr.dtype)
print(edge_smooth_arr.shape)
print(np.amax(edge_smooth_arr))
print(np.amin(edge_smooth_arr))
print(np.mean(edge_smooth_arr))

edge_smooth_arr = np.clip(edge_smooth_arr,np.amin(edge_smooth_arr),1.0)

print("-----")
print(edge_smooth_arr.dtype)
print(edge_smooth_arr.shape)
print(np.amax(edge_smooth_arr))
print(np.amin(edge_smooth_arr))
print(np.mean(edge_smooth_arr))

In [ ]:
scipy.io.savemat(os.path.join(MatlabFilesDir,'edgemask_Q75_Sig2.mat'), mdict={'edgemask': edge_smooth_arr})

# Create edge map image from T1/T2 based on traditional method

In [ ]:
MGI = GenerateSummedGradientImage() # Maximum Gradient Image
MGI.inputs.inputVolume1 = input_hcp_t1
MGI.inputs.inputVolume2 = input_hcp_t2
MGI.inputs.MaximumGradient = True
MGI.inputs.outputFileName = os.path.join(OutputDir,'MaximumGradientImage.nii.gz')
print(MGI.cmdline)
MGI.run()

In [ ]:
mgi = sitk.ReadImage(MGI.inputs.outputFileName)
myshow(mgi)

totalStats = sitk.StatisticsImageFilter()
totalStats.Execute(mgi)
print(totalStats.GetMaximum())
print(totalStats.GetMinimum())

In [ ]:
mgi = sitk.Cast(mgi,sitk.sitkFloat32)
div = sitk.DivideImageFilter()
edgeMask = div.Execute(1.0,mgi)
myshow(edgeMask)

totalStats = sitk.StatisticsImageFilter()
totalStats.Execute(edgeMask)
print(totalStats.GetMaximum())
print(totalStats.GetMinimum())

In [ ]:
sitk.WriteImage(edgeMask,os.path.join(OutputDir,'edgeMask.nii.gz'))

In [ ]:
edge_arr2 = sitk.GetArrayFromImage(edgeMask)

import scipy.io
scipy.io.savemat(os.path.join(MatlabFilesDir,'edgemask_old.mat'), mdict={'edgemask': edge_arr2})

# Extract dwi_b0 image

In [ ]:
dwi_data,dwi_header,bvecs,bvals,gradient_index = ReadNAMICDWIFromNrrd(input_hcp_dwi)

In [ ]:
print(gradient_index)

In [ ]:
print(dwi_data.shape)
print(dwi_data.shape[gradient_index])

In [ ]:
print(dwi_header)

In [ ]:
# Now extract b0 component of the 4D DWI array
dwi_b0 = dwi_data[:,:,:,0]
print(dwi_b0.shape)

To convert the above numpy array to a b0 image, we need careful attention to the order of index and dimensions.

ITK's Image class does not have a bracket operator. It has a GetPixel which takes an ITK Index object as an argument, which is an array ordered as (x,y,z). This is the convention that SimpleITK's Image class uses for the GetPixel method as well.

While in numpy, an array is indexed in the opposite order (z,y,x).

In [ ]:
dwi_b0 = np.transpose(dwi_b0,(2, 1, 0))

In [ ]:
print(dwi_b0.dtype)
print(dwi_b0.shape)
print(np.amax(dwi_b0))
print(np.amin(dwi_b0))
print(np.mean(dwi_b0))

In [ ]:
dwi_b0_img = sitk.GetImageFromArray(dwi_b0)
print(dwi_b0_img.GetSize())
myshow(dwi_b0_img)

In [ ]:
# First read input DWI to get direction and origin
in_dwi = sitk.ReadImage(input_hcp_dwi)

dwi_b0_img = sitk.GetImageFromArray(dwi_b0)
dwi_b0_img.SetOrigin(in_dwi.GetOrigin())
dwi_b0_img.SetDirection(in_dwi.GetDirection())
dwi_b0_img.SetSpacing(in_dwi.GetSpacing())

print(dwi_b0_img.GetSize())
myshow(dwi_b0_img)
sitk.WriteImage(dwi_b0_img,os.path.join(OutputDir,'dwi_b0_normalized.nrrd'))

In [ ]:
# Write the normalized array to a matlab file
#
scipy.io.savemat(os.path.join(MatlabFilesDir,'dwi_b0.mat'), mdict={'inputImage': dwi_b0})

# Extract dwi_b0 image in a traditional way

In [ ]:
dwi_fn='/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/DWI_corrected_alignedSpace.nrrd'

# Now we read dwi image and extract its b0
myExtracter = extractNrrdVectorIndex()
myExtracter.inputs.inputVolume = dwi_fn
myExtracter.inputs.vectorIndex = 0
myExtracter.inputs.outputVolume = os.path.join(OutputDir,'dwi_b0_extracted.nrrd')
print(myExtracter.cmdline)
myExtracter.run()

In [ ]:
dwi_b0 = sitk.ReadImage(myExtracter.inputs.outputVolume)
myshow(dwi_b0)

In [ ]:
def NormalizeBetweenZeroAndOne(image):
    image_double = sitk.Cast(image, sitk.sitkFloat32)
    # new range
    newMax=1.0
    newMin=0
    # Find old range
    totalStats = sitk.StatisticsImageFilter()
    totalStats.Execute(image_double)
    oldMax=totalStats.GetMaximum()
    oldMin=totalStats.GetMinimum()
    f=(newMax-newMin)/(oldMax-oldMin)
    Normalized_image = (image_double-oldMin)*f+newMin
    return Normalized_image

In [ ]:
# Now normalize dwi_b0 to have values between 0 and 1
dwi_b0 = NormalizeBetweenZeroAndOne(dwi_b0)
dwi_im_arr = sitk.GetArrayFromImage(dwi_b0)

import scipy.io
scipy.io.savemat(os.path.join(MatlabFilesDir,'dwib0_traditionalway.mat'), mdict={'inputImage': dwi_im_arr})

In [ ]:
# Normalize array between zero and one
def NormalizeBetweenZeroAndOne(arr):   
    newMax = 1.0
    newMin = 0.0
    #
    oldMax = float(np.amax(arr))
    oldMin = float(np.amin(arr))
    #
    f=(newMax-newMin)/(oldMax-oldMin)
    normalized_arr = (arr-oldMin)*f+newMin
    return normalized_arr

dwi_b0_normalized = NormalizeBetweenZeroAndOne(dwi_b0)

In [ ]:
'''
print(dwi_b0_normalized.dtype)
print(dwi_b0_normalized.shape)
print(np.amax(dwi_b0_normalized))
print(np.amin(dwi_b0_normalized))
print(np.mean(dwi_b0_normalized))
'''

In [ ]:
# Write the normalized array to a matlab file
#
scipy.io.savemat(os.path.join(MatlabFilesDir,'dwi_b0.mat'), mdict={'inputImage': dwi_b0_normalized})

Note that if you want to write the extracted b0 image to disk, you need to get correct "origin", "spacing" and "direction cosign" from the input DWI data.

In [ ]:
# Show b0 image
'''
# First read input DWI to get direction and origin
in_dwi = sitk.ReadImage(input_hcp_dwi)

dwi_b0_img = sitk.GetImageFromArray(dwi_b0)
dwi_b0_img.SetOrigin(in_dwi.GetOrigin())
dwi_b0_img.SetDirection(in_dwi.GetDirection())
dwi_b0_img.SetSpacing(in_dwi.GetSpacing())

print(dwi_b0_img.GetSize())
myshow(dwi_b0_img)
sitk.WriteImage(dwi_b0_img,os.path.join(OutputDir,'dwi_b0.nrrd'))
'''

Also, note that after permutation of dwi_data, now dipy can be used to create the correct tensor model for DWI data. Following runs a test and shows the FA image.

This step is not needed for the purpose of this ticket; however, it will be needed when we are going to compute distance images from the HCP baseline data and the reconstructed images.

In [ ]:
# Compute tensorfit
'''
from dipy.core.gradients import gradient_table
gtab = gradient_table(bvals, bvecs)

from dipy.reconst.dti import TensorModel
ten = TensorModel(gtab)
tenfit = ten.fit(dwi_data)
'''

In [ ]:
# Show FA image
'''
fa = tenfit.fa
fa = np.transpose(fa,(2, 1, 0)) # the reason is inconsistency between index order in numpy and ITK

FA = sitk.GetImageFromArray(fa)
FA.SetOrigin(in_dwi.GetOrigin())
FA.SetDirection(in_dwi.GetDirection())
FA.SetSpacing(in_dwi.GetSpacing())

maskf64 = sitk.Cast(brainmask, sitk.sitkFloat64)
FA = sitk.Multiply(FA,maskf64)

myshow(FA)
sitk.WriteImage(FA,os.path.join(OutputDir,'FA.nrrd'))
'''

# Run 3D SuperResolution from Matlab on Each component